## **Importing Libraries**

In [2]:
import glob
import unicodedata
import string
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Data preparation

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters)

def findFiles(path): return glob.glob(path)

# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = np.zeros((len(line), 1, n_letters))
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

# Build the category_lines dictionary, a list of lines per category
category_lines = {}
all_categories = []
for filename in findFiles('/content/drive/MyDrive/names_data/names/*.txt'):
    category = filename.split("/")[-1].split('.')[0].replace('names\\','')
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]


def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = np.array([all_categories.index(category)])
    line_tensor = lineToTensor(line.lower())
    return category, line, category_tensor, line_tensor

def test():
    for i in range(10):
        category, line, category_tensor, line_tensor = randomTrainingExample()
        print('category =', category, '/ line =', line)

test()

category = Portuguese / line = Torres
category = Italian / line = Sciacca
category = German / line = Senft
category = Spanish / line = Garza
category = Arabic / line = Shadid
category = Italian / line = Aquila
category = Vietnamese / line = La
category = Japanese / line = Nakazawa
category = Greek / line = Houlis
category = Portuguese / line = Santiago


# Generating Trainging Samples

In [5]:
train_lines = []
train_target = []

def generatedata():
  for cate in all_categories:
    for line in category_lines[cate]:
      train_lines.append(lineToTensor(line.lower()).sum(0))
      train_target.append(all_categories.index(cate))

generatedata()

In [6]:
train_lines = np.array(train_lines)
train_target = np.array(train_target)
train_lines = train_lines.squeeze(axis=1)

# Traing Test Split

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_lines, train_target, test_size=0.20, random_state=11,stratify = train_target)

# Build Model

In [8]:
from tensorflow.python.eager.monitoring import Metric

# Determine the input dimension based on the shape of the training data
input_dim = X_train.shape[1]

# Function to build the model based on the specified number of layers, hidden units, and input dimension
def build_model(n_layers, hidden_units, input_dim):
    inp = tf.keras.Input((input_dim,))
    x = inp
    for i in range(n_layers):
        x = tf.keras.layers.Dense(hidden_units, activation='relu')(x)

    x = tf.keras.layers.Dense(18, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=inp, outputs=x)

    opt = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['sparse_categorical_accuracy'])
    return model

# Custom callback class to save the model at specified epochs
class CustomSaver(tf.keras.callbacks.Callback):
    def __init__(self, epochs, n_layers, hidden_units):
        super(CustomSaver, self).__init__()
        self.epochs = epochs
        self.n_layers = n_layers
        self.hidden_units = hidden_units

    def on_epoch_end(self, epoch, logs={}):
        if epoch+1 == int(self.epochs/3) or epoch+1 == int(2*self.epochs/3) or epoch+1 == self.epochs:
            self.model.save(f'model_{self.n_layers}_layers_{self.hidden_units}_units_epoch_{epoch+1}.h5')

# Function to train the model and return the trained model and training history
def train_model(model, X_train, y_train, X_test, y_test, n_layers, hidden_units, epochs):
    # Custom callback to save the model at specified epochs
    model_checkpoint_callback = CustomSaver(epochs, n_layers, hidden_units)

    # Callback to adjust learning rate based on validation accuracy
    lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_sparse_categorical_accuracy', factor=0.001, patience=10)

    # Callback to stop training early if validation accuracy does not improve
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=20)

    # Train the model
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_data=(X_test, y_test), callbacks=[model_checkpoint_callback, lr_schedule, early_stopping])

    return model, history

# Model 1

In [9]:
model1 = build_model(1, 20, input_dim)
train_model(model1, X_train, y_train, X_test, y_test, 1, 20,10)
model1.summary()

Epoch 1/10
502/502 [==============================] - 5s 7ms/step - loss: 1.2829 - sparse_categorical_accuracy: 0.6117 - val_loss: 1.1500 - val_sparse_categorical_accuracy: 0.6491 - lr: 0.0100
Epoch 2/10
502/502 [==============================] - 3s 7ms/step - loss: 1.0797 - sparse_categorical_accuracy: 0.6688 - val_loss: 1.0769 - val_sparse_categorical_accuracy: 0.6677 - lr: 0.0100
Epoch 3/10
502/502 [==============================] - 2s 4ms/step - loss: 1.0280 - sparse_categorical_accuracy: 0.6811 - val_loss: 1.0469 - val_sparse_categorical_accuracy: 0.6857 - lr: 0.0100
Epoch 4/10
502/502 [==============================] - 2s 4ms/step - loss: 1.0017 - sparse_categorical_accuracy: 0.6920 - val_loss: 1.0468 - val_sparse_categorical_accuracy: 0.6625 - lr: 0.0100
Epoch 5/10
502/502 [==============================] - 2s 4ms/step - loss: 0.9853 - sparse_categorical_accuracy: 0.6944 - val_loss: 1.0080 - val_sparse_categorical_accuracy: 0.6929 - lr: 0.0100
Epoch 6/10
502/502 [===============

# Model 2

In [10]:
model2 = build_model(1, 20, input_dim)
train_model(model2, X_train, y_train, X_test, y_test, 1, 20,50)

Epoch 1/50
502/502 [==============================] - 2s 2ms/step - loss: 1.3203 - sparse_categorical_accuracy: 0.6052 - val_loss: 1.1802 - val_sparse_categorical_accuracy: 0.6237 - lr: 0.0100
Epoch 2/50
502/502 [==============================] - 1s 2ms/step - loss: 1.1028 - sparse_categorical_accuracy: 0.6612 - val_loss: 1.1016 - val_sparse_categorical_accuracy: 0.6570 - lr: 0.0100
Epoch 3/50
502/502 [==============================] - 1s 2ms/step - loss: 1.0578 - sparse_categorical_accuracy: 0.6735 - val_loss: 1.0536 - val_sparse_categorical_accuracy: 0.6742 - lr: 0.0100
Epoch 4/50
502/502 [==============================] - 1s 2ms/step - loss: 1.0245 - sparse_categorical_accuracy: 0.6857 - val_loss: 1.0509 - val_sparse_categorical_accuracy: 0.6797 - lr: 0.0100
Epoch 5/50
502/502 [==============================] - 1s 2ms/step - loss: 1.0050 - sparse_categorical_accuracy: 0.6904 - val_loss: 1.0264 - val_sparse_categorical_accuracy: 0.6797 - lr: 0.0100
Epoch 6/50
502/502 [===============

(<keras.engine.functional.Functional at 0x7f711532bdc0>,
 <keras.callbacks.History at 0x7f7111483af0>)

# Model 3

In [11]:
model3 = build_model(1, 20, input_dim)
train_model(model3, X_train, y_train, X_test, y_test, 1, 20,100)
model3.summary()

Epoch 1/100
502/502 [==============================] - 3s 5ms/step - loss: 1.2747 - sparse_categorical_accuracy: 0.6175 - val_loss: 1.1428 - val_sparse_categorical_accuracy: 0.6468 - lr: 0.0100
Epoch 2/100
502/502 [==============================] - 2s 3ms/step - loss: 1.0731 - sparse_categorical_accuracy: 0.6701 - val_loss: 1.0692 - val_sparse_categorical_accuracy: 0.6623 - lr: 0.0100
Epoch 3/100
502/502 [==============================] - 2s 4ms/step - loss: 1.0195 - sparse_categorical_accuracy: 0.6851 - val_loss: 1.0520 - val_sparse_categorical_accuracy: 0.6645 - lr: 0.0100
Epoch 4/100
502/502 [==============================] - 1s 3ms/step - loss: 0.9876 - sparse_categorical_accuracy: 0.6956 - val_loss: 1.0373 - val_sparse_categorical_accuracy: 0.6792 - lr: 0.0100
Epoch 5/100
502/502 [==============================] - 2s 3ms/step - loss: 0.9671 - sparse_categorical_accuracy: 0.7008 - val_loss: 1.0033 - val_sparse_categorical_accuracy: 0.6907 - lr: 0.0100
Epoch 6/100
502/502 [=========

# Model 4

In [12]:
model4 = build_model(1, 50, input_dim)
train_model(model4, X_train, y_train, X_test, y_test, 1, 50,10)
model4.summary()

Epoch 1/10
502/502 [==============================] - 2s 3ms/step - loss: 1.2031 - sparse_categorical_accuracy: 0.6396 - val_loss: 1.0760 - val_sparse_categorical_accuracy: 0.6695 - lr: 0.0100
Epoch 2/10
502/502 [==============================] - 1s 2ms/step - loss: 0.9955 - sparse_categorical_accuracy: 0.6976 - val_loss: 0.9912 - val_sparse_categorical_accuracy: 0.7001 - lr: 0.0100
Epoch 3/10
502/502 [==============================] - 1s 3ms/step - loss: 0.9298 - sparse_categorical_accuracy: 0.7167 - val_loss: 0.9659 - val_sparse_categorical_accuracy: 0.7078 - lr: 0.0100
Epoch 4/10
502/502 [==============================] - 2s 4ms/step - loss: 0.9006 - sparse_categorical_accuracy: 0.7217 - val_loss: 0.9432 - val_sparse_categorical_accuracy: 0.7118 - lr: 0.0100
Epoch 5/10
502/502 [==============================] - 2s 3ms/step - loss: 0.8748 - sparse_categorical_accuracy: 0.7309 - val_loss: 0.9411 - val_sparse_categorical_accuracy: 0.7128 - lr: 0.0100
Epoch 6/10
502/502 [===============

# Model 5

In [13]:
model5 = build_model(1, 50, input_dim)
train_model(model5, X_train, y_train, X_test, y_test, 1, 50,50)
model5.summary()

Epoch 1/50
502/502 [==============================] - 2s 2ms/step - loss: 1.2166 - sparse_categorical_accuracy: 0.6327 - val_loss: 1.0886 - val_sparse_categorical_accuracy: 0.6630 - lr: 0.0100
Epoch 2/50
502/502 [==============================] - 1s 2ms/step - loss: 1.0076 - sparse_categorical_accuracy: 0.6904 - val_loss: 0.9930 - val_sparse_categorical_accuracy: 0.6996 - lr: 0.0100
Epoch 3/50
502/502 [==============================] - 2s 4ms/step - loss: 0.9419 - sparse_categorical_accuracy: 0.7127 - val_loss: 0.9784 - val_sparse_categorical_accuracy: 0.7014 - lr: 0.0100
Epoch 4/50
502/502 [==============================] - 3s 5ms/step - loss: 0.9060 - sparse_categorical_accuracy: 0.7232 - val_loss: 0.9493 - val_sparse_categorical_accuracy: 0.7198 - lr: 0.0100
Epoch 5/50
502/502 [==============================] - 2s 4ms/step - loss: 0.8787 - sparse_categorical_accuracy: 0.7300 - val_loss: 0.9436 - val_sparse_categorical_accuracy: 0.7098 - lr: 0.0100
Epoch 6/50
502/502 [===============

# Model 6

In [14]:
model6 = build_model(1, 50, input_dim)
train_model(model6, X_train, y_train, X_test, y_test, 1, 50,100)

Epoch 1/100
502/502 [==============================] - 2s 3ms/step - loss: 1.2074 - sparse_categorical_accuracy: 0.6380 - val_loss: 1.0739 - val_sparse_categorical_accuracy: 0.6573 - lr: 0.0100
Epoch 2/100
502/502 [==============================] - 1s 2ms/step - loss: 0.9932 - sparse_categorical_accuracy: 0.6974 - val_loss: 0.9816 - val_sparse_categorical_accuracy: 0.6966 - lr: 0.0100
Epoch 3/100
502/502 [==============================] - 1s 2ms/step - loss: 0.9263 - sparse_categorical_accuracy: 0.7147 - val_loss: 0.9989 - val_sparse_categorical_accuracy: 0.7016 - lr: 0.0100
Epoch 4/100
502/502 [==============================] - 1s 2ms/step - loss: 0.8950 - sparse_categorical_accuracy: 0.7246 - val_loss: 0.9497 - val_sparse_categorical_accuracy: 0.7056 - lr: 0.0100
Epoch 5/100
502/502 [==============================] - 1s 2ms/step - loss: 0.8650 - sparse_categorical_accuracy: 0.7329 - val_loss: 0.9422 - val_sparse_categorical_accuracy: 0.7081 - lr: 0.0100
Epoch 6/100
502/502 [=========

(<keras.engine.functional.Functional at 0x7f7111a1e3e0>,
 <keras.callbacks.History at 0x7f7111a7b580>)

# Model 7

In [15]:
model7 = build_model(1, 100, input_dim)
train_model(model7, X_train, y_train, X_test, y_test, 1, 100,10)
model7.summary()

Epoch 1/10
502/502 [==============================] - 2s 3ms/step - loss: 1.1761 - sparse_categorical_accuracy: 0.6471 - val_loss: 1.0230 - val_sparse_categorical_accuracy: 0.6859 - lr: 0.0100
Epoch 2/10
502/502 [==============================] - 1s 2ms/step - loss: 0.9554 - sparse_categorical_accuracy: 0.7086 - val_loss: 0.9425 - val_sparse_categorical_accuracy: 0.7136 - lr: 0.0100
Epoch 3/10
502/502 [==============================] - 1s 2ms/step - loss: 0.8844 - sparse_categorical_accuracy: 0.7286 - val_loss: 0.9266 - val_sparse_categorical_accuracy: 0.7183 - lr: 0.0100
Epoch 4/10
502/502 [==============================] - 1s 2ms/step - loss: 0.8467 - sparse_categorical_accuracy: 0.7415 - val_loss: 0.9258 - val_sparse_categorical_accuracy: 0.7166 - lr: 0.0100
Epoch 5/10
502/502 [==============================] - 1s 2ms/step - loss: 0.8159 - sparse_categorical_accuracy: 0.7439 - val_loss: 0.9168 - val_sparse_categorical_accuracy: 0.7131 - lr: 0.0100
Epoch 6/10
502/502 [===============

# Model 8

In [16]:
model8 = build_model(1, 100, input_dim)
train_model(model8, X_train, y_train, X_test, y_test, 1, 100,50)

Epoch 1/50
502/502 [==============================] - 2s 3ms/step - loss: 1.1700 - sparse_categorical_accuracy: 0.6494 - val_loss: 1.0065 - val_sparse_categorical_accuracy: 0.6959 - lr: 0.0100
Epoch 2/50
502/502 [==============================] - 1s 3ms/step - loss: 0.9460 - sparse_categorical_accuracy: 0.7125 - val_loss: 0.9701 - val_sparse_categorical_accuracy: 0.6969 - lr: 0.0100
Epoch 3/50
502/502 [==============================] - 1s 3ms/step - loss: 0.8822 - sparse_categorical_accuracy: 0.7278 - val_loss: 0.9859 - val_sparse_categorical_accuracy: 0.6974 - lr: 0.0100
Epoch 4/50
502/502 [==============================] - 1s 3ms/step - loss: 0.8468 - sparse_categorical_accuracy: 0.7383 - val_loss: 0.9247 - val_sparse_categorical_accuracy: 0.7196 - lr: 0.0100
Epoch 5/50
502/502 [==============================] - 1s 2ms/step - loss: 0.8139 - sparse_categorical_accuracy: 0.7459 - val_loss: 0.9515 - val_sparse_categorical_accuracy: 0.7148 - lr: 0.0100
Epoch 6/50
502/502 [===============

(<keras.engine.functional.Functional at 0x7f7111714fd0>,
 <keras.callbacks.History at 0x7f7115509f60>)

# Model 9

In [17]:
model9 = build_model(1, 100, input_dim)
train_model(model9, X_train, y_train, X_test, y_test, 1, 100,100)
model9.summary()

Epoch 1/100
502/502 [==============================] - 2s 3ms/step - loss: 1.1661 - sparse_categorical_accuracy: 0.6503 - val_loss: 1.0210 - val_sparse_categorical_accuracy: 0.6892 - lr: 0.0100
Epoch 2/100
502/502 [==============================] - 1s 2ms/step - loss: 0.9432 - sparse_categorical_accuracy: 0.7144 - val_loss: 0.9742 - val_sparse_categorical_accuracy: 0.7098 - lr: 0.0100
Epoch 3/100
502/502 [==============================] - 1s 3ms/step - loss: 0.8794 - sparse_categorical_accuracy: 0.7306 - val_loss: 0.9494 - val_sparse_categorical_accuracy: 0.7116 - lr: 0.0100
Epoch 4/100
502/502 [==============================] - 1s 3ms/step - loss: 0.8409 - sparse_categorical_accuracy: 0.7405 - val_loss: 0.9151 - val_sparse_categorical_accuracy: 0.7258 - lr: 0.0100
Epoch 5/100
502/502 [==============================] - 2s 3ms/step - loss: 0.8135 - sparse_categorical_accuracy: 0.7466 - val_loss: 0.9209 - val_sparse_categorical_accuracy: 0.7143 - lr: 0.0100
Epoch 6/100
502/502 [=========

# Model 10

In [18]:
model10 = build_model(2, 20, input_dim)
train_model(model10, X_train, y_train, X_test, y_test, 2, 20,10)
model10.summary()

Epoch 1/10
502/502 [==============================] - 2s 2ms/step - loss: 1.3159 - sparse_categorical_accuracy: 0.6144 - val_loss: 1.1892 - val_sparse_categorical_accuracy: 0.6468 - lr: 0.0100
Epoch 2/10
502/502 [==============================] - 1s 2ms/step - loss: 1.1079 - sparse_categorical_accuracy: 0.6680 - val_loss: 1.0828 - val_sparse_categorical_accuracy: 0.6777 - lr: 0.0100
Epoch 3/10
502/502 [==============================] - 1s 2ms/step - loss: 1.0522 - sparse_categorical_accuracy: 0.6786 - val_loss: 1.0874 - val_sparse_categorical_accuracy: 0.6785 - lr: 0.0100
Epoch 4/10
502/502 [==============================] - 1s 2ms/step - loss: 1.0235 - sparse_categorical_accuracy: 0.6885 - val_loss: 1.0306 - val_sparse_categorical_accuracy: 0.6837 - lr: 0.0100
Epoch 5/10
502/502 [==============================] - 1s 3ms/step - loss: 1.0021 - sparse_categorical_accuracy: 0.6956 - val_loss: 1.0727 - val_sparse_categorical_accuracy: 0.6692 - lr: 0.0100
Epoch 6/10
502/502 [===============

# Model 11

In [19]:
model11 = build_model(2, 20, input_dim)
train_model(model11, X_train, y_train, X_test, y_test, 2, 20,50)

Epoch 1/50
502/502 [==============================] - 2s 3ms/step - loss: 1.3201 - sparse_categorical_accuracy: 0.6179 - val_loss: 1.2096 - val_sparse_categorical_accuracy: 0.6356 - lr: 0.0100
Epoch 2/50
502/502 [==============================] - 1s 2ms/step - loss: 1.1014 - sparse_categorical_accuracy: 0.6690 - val_loss: 1.0801 - val_sparse_categorical_accuracy: 0.6807 - lr: 0.0100
Epoch 3/50
502/502 [==============================] - 1s 2ms/step - loss: 1.0358 - sparse_categorical_accuracy: 0.6847 - val_loss: 1.0620 - val_sparse_categorical_accuracy: 0.6675 - lr: 0.0100
Epoch 4/50
502/502 [==============================] - 1s 2ms/step - loss: 0.9988 - sparse_categorical_accuracy: 0.6948 - val_loss: 1.0401 - val_sparse_categorical_accuracy: 0.6914 - lr: 0.0100
Epoch 5/50
502/502 [==============================] - 1s 2ms/step - loss: 0.9842 - sparse_categorical_accuracy: 0.6977 - val_loss: 1.0072 - val_sparse_categorical_accuracy: 0.6951 - lr: 0.0100
Epoch 6/50
502/502 [===============

(<keras.engine.functional.Functional at 0x7f7110fa3bb0>,
 <keras.callbacks.History at 0x7f7110fc6800>)

# Model 12

In [20]:
model12 = build_model(2, 20, input_dim)
train_model(model12, X_train, y_train, X_test, y_test, 2, 20,100)

Epoch 1/100
502/502 [==============================] - 2s 3ms/step - loss: 1.2983 - sparse_categorical_accuracy: 0.6141 - val_loss: 1.1815 - val_sparse_categorical_accuracy: 0.6463 - lr: 0.0100
Epoch 2/100
502/502 [==============================] - 1s 2ms/step - loss: 1.0794 - sparse_categorical_accuracy: 0.6738 - val_loss: 1.0752 - val_sparse_categorical_accuracy: 0.6732 - lr: 0.0100
Epoch 3/100
502/502 [==============================] - 1s 2ms/step - loss: 1.0254 - sparse_categorical_accuracy: 0.6895 - val_loss: 1.0468 - val_sparse_categorical_accuracy: 0.6762 - lr: 0.0100
Epoch 4/100
502/502 [==============================] - 1s 2ms/step - loss: 0.9952 - sparse_categorical_accuracy: 0.6995 - val_loss: 1.0289 - val_sparse_categorical_accuracy: 0.6936 - lr: 0.0100
Epoch 5/100
502/502 [==============================] - 1s 2ms/step - loss: 0.9806 - sparse_categorical_accuracy: 0.7020 - val_loss: 1.0314 - val_sparse_categorical_accuracy: 0.6842 - lr: 0.0100
Epoch 6/100
502/502 [=========

(<keras.engine.functional.Functional at 0x7f7110155d80>,
 <keras.callbacks.History at 0x7f7110116ad0>)

# Model 13

In [21]:
model13 = build_model(2, 50, input_dim)
train_model(model13, X_train, y_train, X_test, y_test, 2, 50,10)
model13.summary()

Epoch 1/10
502/502 [==============================] - 2s 2ms/step - loss: 1.2274 - sparse_categorical_accuracy: 0.6395 - val_loss: 1.1033 - val_sparse_categorical_accuracy: 0.6732 - lr: 0.0100
Epoch 2/10
502/502 [==============================] - 1s 2ms/step - loss: 1.0001 - sparse_categorical_accuracy: 0.7009 - val_loss: 1.0060 - val_sparse_categorical_accuracy: 0.6971 - lr: 0.0100
Epoch 3/10
502/502 [==============================] - 1s 2ms/step - loss: 0.9275 - sparse_categorical_accuracy: 0.7179 - val_loss: 1.0100 - val_sparse_categorical_accuracy: 0.7066 - lr: 0.0100
Epoch 4/10
502/502 [==============================] - 1s 2ms/step - loss: 0.8990 - sparse_categorical_accuracy: 0.7269 - val_loss: 0.9641 - val_sparse_categorical_accuracy: 0.7078 - lr: 0.0100
Epoch 5/10
502/502 [==============================] - 1s 2ms/step - loss: 0.8721 - sparse_categorical_accuracy: 0.7329 - val_loss: 0.9765 - val_sparse_categorical_accuracy: 0.7148 - lr: 0.0100
Epoch 6/10
502/502 [===============

# Model 14

In [22]:
model14 = build_model(2, 50, input_dim)
train_model(model14, X_train, y_train, X_test, y_test, 2, 50,50)

Epoch 1/50
502/502 [==============================] - 2s 2ms/step - loss: 1.2127 - sparse_categorical_accuracy: 0.6399 - val_loss: 1.0661 - val_sparse_categorical_accuracy: 0.6765 - lr: 0.0100
Epoch 2/50
502/502 [==============================] - 1s 2ms/step - loss: 0.9933 - sparse_categorical_accuracy: 0.7004 - val_loss: 1.0128 - val_sparse_categorical_accuracy: 0.7021 - lr: 0.0100
Epoch 3/50
502/502 [==============================] - 1s 2ms/step - loss: 0.9272 - sparse_categorical_accuracy: 0.7151 - val_loss: 0.9853 - val_sparse_categorical_accuracy: 0.7051 - lr: 0.0100
Epoch 4/50
502/502 [==============================] - 1s 2ms/step - loss: 0.8880 - sparse_categorical_accuracy: 0.7302 - val_loss: 0.9952 - val_sparse_categorical_accuracy: 0.7118 - lr: 0.0100
Epoch 5/50
502/502 [==============================] - 1s 2ms/step - loss: 0.8626 - sparse_categorical_accuracy: 0.7350 - val_loss: 0.9465 - val_sparse_categorical_accuracy: 0.7086 - lr: 0.0100
Epoch 6/50
502/502 [===============

(<keras.engine.functional.Functional at 0x7f70f9da8250>,
 <keras.callbacks.History at 0x7f70f9dab370>)

# Model 15

In [23]:
m15 = build_model(2, 50, input_dim)
train_model(m15, X_train, y_train, X_test, y_test, 2, 50,100)

Epoch 1/100
502/502 [==============================] - 2s 3ms/step - loss: 1.2302 - sparse_categorical_accuracy: 0.6357 - val_loss: 1.1674 - val_sparse_categorical_accuracy: 0.6324 - lr: 0.0100
Epoch 2/100
502/502 [==============================] - 1s 2ms/step - loss: 1.0044 - sparse_categorical_accuracy: 0.6990 - val_loss: 1.0002 - val_sparse_categorical_accuracy: 0.6981 - lr: 0.0100
Epoch 3/100
502/502 [==============================] - 1s 2ms/step - loss: 0.9367 - sparse_categorical_accuracy: 0.7125 - val_loss: 0.9905 - val_sparse_categorical_accuracy: 0.6956 - lr: 0.0100
Epoch 4/100
502/502 [==============================] - 1s 2ms/step - loss: 0.9016 - sparse_categorical_accuracy: 0.7250 - val_loss: 0.9815 - val_sparse_categorical_accuracy: 0.7101 - lr: 0.0100
Epoch 5/100
502/502 [==============================] - 1s 2ms/step - loss: 0.8713 - sparse_categorical_accuracy: 0.7288 - val_loss: 0.9518 - val_sparse_categorical_accuracy: 0.7064 - lr: 0.0100
Epoch 6/100
502/502 [=========

(<keras.engine.functional.Functional at 0x7f70f9afee00>,
 <keras.callbacks.History at 0x7f70f9b27610>)

# Model 16

In [24]:
model16 = build_model(2, 100, input_dim)
train_model(model16, X_train, y_train, X_test, y_test, 2, 100,10)
model16.summary()

Epoch 1/10
502/502 [==============================] - 2s 2ms/step - loss: 1.1918 - sparse_categorical_accuracy: 0.6482 - val_loss: 1.0666 - val_sparse_categorical_accuracy: 0.6670 - lr: 0.0100
Epoch 2/10
502/502 [==============================] - 1s 2ms/step - loss: 0.9720 - sparse_categorical_accuracy: 0.7055 - val_loss: 1.0141 - val_sparse_categorical_accuracy: 0.6979 - lr: 0.0100
Epoch 3/10
502/502 [==============================] - 1s 2ms/step - loss: 0.8996 - sparse_categorical_accuracy: 0.7255 - val_loss: 0.9676 - val_sparse_categorical_accuracy: 0.7126 - lr: 0.0100
Epoch 4/10
502/502 [==============================] - 1s 2ms/step - loss: 0.8550 - sparse_categorical_accuracy: 0.7343 - val_loss: 0.9627 - val_sparse_categorical_accuracy: 0.7200 - lr: 0.0100
Epoch 5/10
502/502 [==============================] - 2s 3ms/step - loss: 0.8184 - sparse_categorical_accuracy: 0.7437 - val_loss: 0.9634 - val_sparse_categorical_accuracy: 0.7131 - lr: 0.0100
Epoch 6/10
502/502 [===============

# Model 17

In [25]:
model17 = build_model(2, 100, input_dim)
train_model(model17, X_train, y_train, X_test, y_test, 2, 100,50)

Epoch 1/50
502/502 [==============================] - 2s 2ms/step - loss: 1.1973 - sparse_categorical_accuracy: 0.6460 - val_loss: 1.0653 - val_sparse_categorical_accuracy: 0.6817 - lr: 0.0100
Epoch 2/50
502/502 [==============================] - 1s 2ms/step - loss: 0.9732 - sparse_categorical_accuracy: 0.7076 - val_loss: 0.9550 - val_sparse_categorical_accuracy: 0.7128 - lr: 0.0100
Epoch 3/50
502/502 [==============================] - 1s 2ms/step - loss: 0.8960 - sparse_categorical_accuracy: 0.7246 - val_loss: 0.9597 - val_sparse_categorical_accuracy: 0.7131 - lr: 0.0100
Epoch 4/50
502/502 [==============================] - 1s 2ms/step - loss: 0.8507 - sparse_categorical_accuracy: 0.7358 - val_loss: 0.9805 - val_sparse_categorical_accuracy: 0.7111 - lr: 0.0100
Epoch 5/50
502/502 [==============================] - 1s 3ms/step - loss: 0.8242 - sparse_categorical_accuracy: 0.7444 - val_loss: 0.9651 - val_sparse_categorical_accuracy: 0.7096 - lr: 0.0100
Epoch 6/50
502/502 [===============

(<keras.engine.functional.Functional at 0x7f70f9ad9960>,
 <keras.callbacks.History at 0x7f70f98778e0>)

# Model 18

In [26]:
model18 = build_model(2, 100, input_dim)
train_model(model18, X_train, y_train, X_test, y_test, 2, 100,100)

Epoch 1/100
502/502 [==============================] - 2s 3ms/step - loss: 1.2032 - sparse_categorical_accuracy: 0.6459 - val_loss: 1.0553 - val_sparse_categorical_accuracy: 0.6879 - lr: 0.0100
Epoch 2/100
502/502 [==============================] - 1s 2ms/step - loss: 0.9832 - sparse_categorical_accuracy: 0.7066 - val_loss: 1.0231 - val_sparse_categorical_accuracy: 0.6862 - lr: 0.0100
Epoch 3/100
502/502 [==============================] - 1s 2ms/step - loss: 0.9047 - sparse_categorical_accuracy: 0.7220 - val_loss: 0.9600 - val_sparse_categorical_accuracy: 0.7123 - lr: 0.0100
Epoch 4/100
502/502 [==============================] - 1s 2ms/step - loss: 0.8533 - sparse_categorical_accuracy: 0.7363 - val_loss: 0.9470 - val_sparse_categorical_accuracy: 0.7153 - lr: 0.0100
Epoch 5/100
502/502 [==============================] - 1s 2ms/step - loss: 0.8210 - sparse_categorical_accuracy: 0.7440 - val_loss: 0.9276 - val_sparse_categorical_accuracy: 0.7183 - lr: 0.0100
Epoch 6/100
502/502 [=========

(<keras.engine.functional.Functional at 0x7f70f47d99f0>,
 <keras.callbacks.History at 0x7f70f4600f70>)

# Model 19

In [27]:
model19 = build_model(3, 20, input_dim)
train_model(model19, X_train, y_train, X_test, y_test, 3, 50,10)
model19.summary()

Epoch 1/10
502/502 [==============================] - 3s 4ms/step - loss: 1.3594 - sparse_categorical_accuracy: 0.6063 - val_loss: 1.2291 - val_sparse_categorical_accuracy: 0.6438 - lr: 0.0100
Epoch 2/10
502/502 [==============================] - 1s 3ms/step - loss: 1.1651 - sparse_categorical_accuracy: 0.6637 - val_loss: 1.1443 - val_sparse_categorical_accuracy: 0.6692 - lr: 0.0100
Epoch 3/10
502/502 [==============================] - 1s 2ms/step - loss: 1.1006 - sparse_categorical_accuracy: 0.6779 - val_loss: 1.1018 - val_sparse_categorical_accuracy: 0.6842 - lr: 0.0100
Epoch 4/10
502/502 [==============================] - 1s 2ms/step - loss: 1.0571 - sparse_categorical_accuracy: 0.6892 - val_loss: 1.0935 - val_sparse_categorical_accuracy: 0.6819 - lr: 0.0100
Epoch 5/10
502/502 [==============================] - 1s 2ms/step - loss: 1.0306 - sparse_categorical_accuracy: 0.6946 - val_loss: 1.0450 - val_sparse_categorical_accuracy: 0.6994 - lr: 0.0100
Epoch 6/10
502/502 [===============

# Model 20

In [28]:
model20 = build_model(3, 20, input_dim)
train_model(model20, X_train, y_train, X_test, y_test, 3, 20,50)

Epoch 1/50
502/502 [==============================] - 2s 2ms/step - loss: 1.3672 - sparse_categorical_accuracy: 0.6023 - val_loss: 1.2208 - val_sparse_categorical_accuracy: 0.6471 - lr: 0.0100
Epoch 2/50
502/502 [==============================] - 1s 2ms/step - loss: 1.1352 - sparse_categorical_accuracy: 0.6649 - val_loss: 1.1361 - val_sparse_categorical_accuracy: 0.6790 - lr: 0.0100
Epoch 3/50
502/502 [==============================] - 1s 2ms/step - loss: 1.0651 - sparse_categorical_accuracy: 0.6807 - val_loss: 1.0855 - val_sparse_categorical_accuracy: 0.6887 - lr: 0.0100
Epoch 4/50
502/502 [==============================] - 1s 2ms/step - loss: 1.0301 - sparse_categorical_accuracy: 0.6924 - val_loss: 1.0439 - val_sparse_categorical_accuracy: 0.6946 - lr: 0.0100
Epoch 5/50
502/502 [==============================] - 2s 3ms/step - loss: 1.0085 - sparse_categorical_accuracy: 0.7003 - val_loss: 1.0370 - val_sparse_categorical_accuracy: 0.6914 - lr: 0.0100
Epoch 6/50
502/502 [===============

(<keras.engine.functional.Functional at 0x7f70f432e680>,
 <keras.callbacks.History at 0x7f70f4359c00>)

# Model 21

In [29]:
model21 = build_model(3, 20, input_dim)
train_model(model21, X_train, y_train, X_test, y_test, 3, 20,100)

Epoch 1/100
502/502 [==============================] - 2s 2ms/step - loss: 1.3464 - sparse_categorical_accuracy: 0.6136 - val_loss: 1.1969 - val_sparse_categorical_accuracy: 0.6528 - lr: 0.0100
Epoch 2/100
502/502 [==============================] - 1s 2ms/step - loss: 1.1383 - sparse_categorical_accuracy: 0.6616 - val_loss: 1.1043 - val_sparse_categorical_accuracy: 0.6777 - lr: 0.0100
Epoch 3/100
502/502 [==============================] - 1s 2ms/step - loss: 1.0568 - sparse_categorical_accuracy: 0.6842 - val_loss: 1.0785 - val_sparse_categorical_accuracy: 0.6834 - lr: 0.0100
Epoch 4/100
502/502 [==============================] - 1s 2ms/step - loss: 1.0254 - sparse_categorical_accuracy: 0.6928 - val_loss: 1.0823 - val_sparse_categorical_accuracy: 0.6780 - lr: 0.0100
Epoch 5/100
502/502 [==============================] - 2s 3ms/step - loss: 1.0051 - sparse_categorical_accuracy: 0.7002 - val_loss: 1.0552 - val_sparse_categorical_accuracy: 0.6869 - lr: 0.0100
Epoch 6/100
502/502 [=========

(<keras.engine.functional.Functional at 0x7f70f98765f0>,
 <keras.callbacks.History at 0x7f70f98747f0>)

# Model 22

In [30]:
model22 = build_model(3, 50, input_dim)
train_model(model22, X_train, y_train, X_test, y_test, 3, 50,10)
model22.summary()

Epoch 1/10
502/502 [==============================] - 2s 2ms/step - loss: 1.3076 - sparse_categorical_accuracy: 0.6265 - val_loss: 1.1327 - val_sparse_categorical_accuracy: 0.6720 - lr: 0.0100
Epoch 2/10
502/502 [==============================] - 1s 2ms/step - loss: 1.0547 - sparse_categorical_accuracy: 0.6844 - val_loss: 1.0704 - val_sparse_categorical_accuracy: 0.6932 - lr: 0.0100
Epoch 3/10
502/502 [==============================] - 2s 3ms/step - loss: 0.9720 - sparse_categorical_accuracy: 0.7099 - val_loss: 0.9969 - val_sparse_categorical_accuracy: 0.7049 - lr: 0.0100
Epoch 4/10
502/502 [==============================] - 2s 3ms/step - loss: 0.9285 - sparse_categorical_accuracy: 0.7185 - val_loss: 1.0011 - val_sparse_categorical_accuracy: 0.7039 - lr: 0.0100
Epoch 5/10
502/502 [==============================] - 1s 3ms/step - loss: 0.9046 - sparse_categorical_accuracy: 0.7241 - val_loss: 1.0307 - val_sparse_categorical_accuracy: 0.6892 - lr: 0.0100
Epoch 6/10
502/502 [===============

# Model 23

In [31]:
model23 = build_model(3, 50, input_dim)
train_model(model23, X_train, y_train, X_test, y_test, 3, 50,50)

Epoch 1/50
502/502 [==============================] - 2s 3ms/step - loss: 1.2776 - sparse_categorical_accuracy: 0.6319 - val_loss: 1.1265 - val_sparse_categorical_accuracy: 0.6735 - lr: 0.0100
Epoch 2/50
502/502 [==============================] - 1s 2ms/step - loss: 1.0592 - sparse_categorical_accuracy: 0.6873 - val_loss: 1.0804 - val_sparse_categorical_accuracy: 0.6785 - lr: 0.0100
Epoch 3/50
502/502 [==============================] - 1s 3ms/step - loss: 0.9731 - sparse_categorical_accuracy: 0.7057 - val_loss: 1.0278 - val_sparse_categorical_accuracy: 0.6857 - lr: 0.0100
Epoch 4/50
502/502 [==============================] - 2s 3ms/step - loss: 0.9395 - sparse_categorical_accuracy: 0.7193 - val_loss: 1.0337 - val_sparse_categorical_accuracy: 0.6864 - lr: 0.0100
Epoch 5/50
502/502 [==============================] - 2s 3ms/step - loss: 0.9142 - sparse_categorical_accuracy: 0.7226 - val_loss: 0.9788 - val_sparse_categorical_accuracy: 0.7078 - lr: 0.0100
Epoch 6/50
502/502 [===============

(<keras.engine.functional.Functional at 0x7f70ebef3070>,
 <keras.callbacks.History at 0x7f70ebf232b0>)

# Model 24

In [32]:
model24 = build_model(3, 50, input_dim)
train_model(model24, X_train, y_train, X_test, y_test, 3, 50,100)

Epoch 1/100
502/502 [==============================] - 2s 3ms/step - loss: 1.2703 - sparse_categorical_accuracy: 0.6304 - val_loss: 1.1468 - val_sparse_categorical_accuracy: 0.6665 - lr: 0.0100
Epoch 2/100
502/502 [==============================] - 1s 2ms/step - loss: 1.0584 - sparse_categorical_accuracy: 0.6896 - val_loss: 1.0817 - val_sparse_categorical_accuracy: 0.6710 - lr: 0.0100
Epoch 3/100
502/502 [==============================] - 1s 2ms/step - loss: 0.9747 - sparse_categorical_accuracy: 0.7085 - val_loss: 1.0125 - val_sparse_categorical_accuracy: 0.7014 - lr: 0.0100
Epoch 4/100
502/502 [==============================] - 1s 2ms/step - loss: 0.9454 - sparse_categorical_accuracy: 0.7165 - val_loss: 1.0193 - val_sparse_categorical_accuracy: 0.7049 - lr: 0.0100
Epoch 5/100
502/502 [==============================] - 2s 3ms/step - loss: 0.9129 - sparse_categorical_accuracy: 0.7257 - val_loss: 0.9988 - val_sparse_categorical_accuracy: 0.6974 - lr: 0.0100
Epoch 6/100
502/502 [=========

(<keras.engine.functional.Functional at 0x7f70edde8b50>,
 <keras.callbacks.History at 0x7f70eddeb670>)

# Model 25

In [33]:
model25 = build_model(3, 100, input_dim)
train_model(model25, X_train, y_train, X_test, y_test, 3, 100,10)
model25.summary()

Epoch 1/10
502/502 [==============================] - 2s 3ms/step - loss: 1.2652 - sparse_categorical_accuracy: 0.6334 - val_loss: 1.1165 - val_sparse_categorical_accuracy: 0.6732 - lr: 0.0100
Epoch 2/10
502/502 [==============================] - 1s 3ms/step - loss: 1.0330 - sparse_categorical_accuracy: 0.6956 - val_loss: 1.0326 - val_sparse_categorical_accuracy: 0.6907 - lr: 0.0100
Epoch 3/10
502/502 [==============================] - 1s 3ms/step - loss: 0.9531 - sparse_categorical_accuracy: 0.7149 - val_loss: 1.0183 - val_sparse_categorical_accuracy: 0.6892 - lr: 0.0100
Epoch 4/10
502/502 [==============================] - 1s 3ms/step - loss: 0.9208 - sparse_categorical_accuracy: 0.7241 - val_loss: 1.0082 - val_sparse_categorical_accuracy: 0.7054 - lr: 0.0100
Epoch 5/10
502/502 [==============================] - 2s 3ms/step - loss: 0.8814 - sparse_categorical_accuracy: 0.7307 - val_loss: 0.9843 - val_sparse_categorical_accuracy: 0.7116 - lr: 0.0100
Epoch 6/10
502/502 [===============

# Model 26

In [34]:
model26 = build_model(3, 100, input_dim)
train_model(model26, X_train, y_train, X_test, y_test, 3, 100,50)

Epoch 1/50
502/502 [==============================] - 3s 3ms/step - loss: 1.2486 - sparse_categorical_accuracy: 0.6385 - val_loss: 1.1065 - val_sparse_categorical_accuracy: 0.6717 - lr: 0.0100
Epoch 2/50
502/502 [==============================] - 1s 2ms/step - loss: 1.0202 - sparse_categorical_accuracy: 0.6988 - val_loss: 1.0473 - val_sparse_categorical_accuracy: 0.6887 - lr: 0.0100
Epoch 3/50
502/502 [==============================] - 1s 2ms/step - loss: 0.9461 - sparse_categorical_accuracy: 0.7154 - val_loss: 0.9825 - val_sparse_categorical_accuracy: 0.7133 - lr: 0.0100
Epoch 4/50
502/502 [==============================] - 1s 3ms/step - loss: 0.9072 - sparse_categorical_accuracy: 0.7271 - val_loss: 0.9950 - val_sparse_categorical_accuracy: 0.6927 - lr: 0.0100
Epoch 5/50
502/502 [==============================] - 1s 2ms/step - loss: 0.8770 - sparse_categorical_accuracy: 0.7356 - val_loss: 0.9761 - val_sparse_categorical_accuracy: 0.7136 - lr: 0.0100
Epoch 6/50
502/502 [===============

(<keras.engine.functional.Functional at 0x7f70edb99d80>,
 <keras.callbacks.History at 0x7f70edcb1990>)

# Model 27

In [35]:
model27 = build_model(3, 100, input_dim)
train_model(model27, X_train, y_train, X_test, y_test, 3, 100,100)

Epoch 1/100
502/502 [==============================] - 2s 3ms/step - loss: 1.2447 - sparse_categorical_accuracy: 0.6441 - val_loss: 1.1280 - val_sparse_categorical_accuracy: 0.6682 - lr: 0.0100
Epoch 2/100
502/502 [==============================] - 1s 2ms/step - loss: 1.0353 - sparse_categorical_accuracy: 0.6996 - val_loss: 1.0225 - val_sparse_categorical_accuracy: 0.6984 - lr: 0.0100
Epoch 3/100
502/502 [==============================] - 1s 3ms/step - loss: 0.9610 - sparse_categorical_accuracy: 0.7126 - val_loss: 0.9992 - val_sparse_categorical_accuracy: 0.6986 - lr: 0.0100
Epoch 4/100
502/502 [==============================] - 2s 3ms/step - loss: 0.9139 - sparse_categorical_accuracy: 0.7283 - val_loss: 0.9902 - val_sparse_categorical_accuracy: 0.7108 - lr: 0.0100
Epoch 5/100
502/502 [==============================] - 2s 4ms/step - loss: 0.8832 - sparse_categorical_accuracy: 0.7342 - val_loss: 1.0476 - val_sparse_categorical_accuracy: 0.6964 - lr: 0.0100
Epoch 6/100
502/502 [=========

(<keras.engine.functional.Functional at 0x7f70ed95b8b0>,
 <keras.callbacks.History at 0x7f70ed982f80>)

In [36]:
from sklearn.metrics import accuracy_score
y_pred = model1.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 1ms/step
The overall accuracy score is 0.6926525529265255


In [37]:
y_pred = model2.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 2ms/step
The overall accuracy score is 0.7026151930261519


In [38]:
y_pred = model3.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 2ms/step
The overall accuracy score is 0.7080946450809464


In [39]:
y_pred = model4.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 2ms/step
The overall accuracy score is 0.7198007471980075


In [40]:
y_pred = model5.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 2ms/step
The overall accuracy score is 0.726027397260274


In [41]:
y_pred = model6.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 2ms/step
The overall accuracy score is 0.7178082191780822


In [42]:
y_pred = model7.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 2ms/step
The overall accuracy score is 0.7287671232876712


In [44]:
y_pred = model8.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 1s 5ms/step
The overall accuracy score is 0.725280199252802


In [45]:
y_pred = model9.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 1ms/step
The overall accuracy score is 0.7295143212951433


In [46]:
y_pred = model10.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 1ms/step
The overall accuracy score is 0.7018679950186799


In [47]:
y_pred = model11.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 1ms/step
The overall accuracy score is 0.710585305105853


In [48]:
y_pred = model12.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 1ms/step
The overall accuracy score is 0.7068493150684931


In [49]:
y_pred = model13.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 1ms/step
The overall accuracy score is 0.7041095890410959


In [50]:
y_pred = model14.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 1ms/step
The overall accuracy score is 0.726027397260274


In [51]:
y_pred = m15.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 2ms/step
The overall accuracy score is 0.7275217932752179


In [52]:
y_pred = model16.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 1ms/step
The overall accuracy score is 0.7153175591531756


In [53]:
y_pred = model17.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 1ms/step
The overall accuracy score is 0.7118306351183064


In [54]:
y_pred = model18.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 1ms/step
The overall accuracy score is 0.7230386052303861


In [55]:
y_pred = model19.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 1ms/step
The overall accuracy score is 0.6946450809464508


In [56]:
y_pred = model20.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 1ms/step
The overall accuracy score is 0.723785803237858


In [57]:
y_pred = model21.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 2ms/step
The overall accuracy score is 0.7048567870485679


In [58]:
y_pred = model22.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 1ms/step
The overall accuracy score is 0.7113325031133251


In [59]:
y_pred = model23.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 1ms/step
The overall accuracy score is 0.7265255292652553


In [60]:
y_pred = model24.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 1ms/step
The overall accuracy score is 0.7227895392278954


In [61]:
y_pred = model25.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 1ms/step
The overall accuracy score is 0.7123287671232876


In [62]:
y_pred = model26.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 1ms/step
The overall accuracy score is 0.7198007471980075


In [63]:
y_pred = model27.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'The overall accuracy score is {accuracy}')

126/126 [==============================] - 0s 2ms/step
The overall accuracy score is 0.7270236612702367



Conclusion:
The task involved training 27 models with different configurations, varying the number of hidden layers, the size of each hidden layer, and the training epochs. The three configurations tested an their accuracy scores for each:

* Configuration with 1 hidden layer:

Hidden layer size: 20, 50, 100

Training epochs: 10, 50, 100

Accuracy score: 0.6926525529265255, 0.7026151930261519, 0.7080946450809464

* Configuration with 2 hidden layers:

Hidden layer size: 20, 50, 100

Training epochs: 10, 50, 100

Accuracy score: 0.7198007471980075, 0.726027397260274, 0.7178082191780822

* Configuration with 3 hidden layers:

Hidden layer size: 20, 50, 100

Training epochs: 10, 50, 100

Accuracy score: 0.7287671232876712, 0.725280199252802, 0.7295143212951433

Based on these results, we can conclude that:

* The highest accuracy score achieved was 0.7295143212951433, which was obtained with the configuration having 3 hidden layers, each with a size of 100, and trained for 10, 50, and 100 epochs.

* The configuration with 2 hidden layers also performed well, with accuracy scores ranging from 0.7118306351183064 to 0.7205479452054795.

* The configuration with 1 hidden layer had slightly lower accuracy scores, ranging from 0.6926525529265255 to 0.7118306351183064.



In general, the accuracy of the models improved as we increased the number of hidden layers and made them larger. However, it can also be that the soultion would depend on the dataset.

